In [256]:
import pandas as pd
import numpy as np

import re
import nltk
import spacy
import tqdm as notebook_tqdm

import warnings
warnings.filterwarnings('ignore')

In [257]:
temp = pd.read_csv("../data/commentory.csv")

In [258]:
temp['comment'] = temp['comment'].apply(lambda x: x.lower())
temp['home_team'] = temp['home_team'].apply(lambda x: x.lower())
temp['away_team'] = temp['away_team'].apply(lambda x: x.lower())

In [259]:
df_unique = pd.DataFrame(temp["match"].unique())
df_unique.reset_index(inplace=True)
df_unique.columns = ["match_id", "match"]
df = pd.merge(temp, df_unique, on = "match")

In [260]:
df_new = pd.DataFrame()

for i in df["match_id"].unique():
    temp = df[df["match_id"] == i]
    temp.reset_index(inplace=True, drop=True)

    home_team = temp["home_team"][0]
    away_team = temp["away_team"][0]

    temp["comment"] = temp["comment"].apply(lambda x: x.strip())

    for i in range(len(temp)):
        if (home_team + " subs" in temp["comment"][i]) | (
            home_team + " subs:" in temp["comment"][i]
        ):
            temp["home_team_subs_info"] = temp["comment"][i]
            temp["home_team_formation_info"] = temp["comment"][i + 1]

        elif (away_team + " subs" in temp["comment"][i]) | (
            away_team + " subs:" in temp["comment"][i]
        ):
            temp["away_team_subs_info"] = temp["comment"][i]
            temp["away_team_formation_info"] = temp["comment"][i + 1]

        else:
            continue

        df_new = df_new.append(temp)
        df_new.reset_index(inplace=True, drop=True)


In [266]:
df_new = pd.read_csv("../data/formation_subs.csv")
df_new.head()

,time,comment,event,event_player,event_team,comment_desc,home_team,home_team_abbr,away_team,away_team_abbr,full_time_score,match,date,link,match_id,home_team_subs_info,home_team_formation_info,away_team_subs_info,away_team_formation_info
0,NaN,thanks for joining our commentary this evening...,NaN,NaN,NaN,full time summary,barcelona,BAR,bayern munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...",barcelona (3-1-4-2): marc-andre ter stegen; ro...,"bayern subs: omar ricards, serge gnabry, kings...",bayern (4-2-3-1): manuel neuer; benjamin pavar...
1,NaN,barcelona are next in action at home in laliga...,NaN,NaN,NaN,full time summary,barcelona,BAR,bayern munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...",barcelona (3-1-4-2): marc-andre ter stegen; ro...,"bayern subs: omar ricards, serge gnabry, kings...",bayern (4-2-3-1): manuel neuer; benjamin pavar...
2,NaN,bayern munich have eased past barcelona in the...,NaN,NaN,NaN,full time summary,barcelona,BAR,bayern munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...",barcelona (3-1-4-2): marc-andre ter stegen; ro...,"bayern subs: omar ricards, serge gnabry, kings...",bayern (4-2-3-1): manuel neuer; benjamin pavar...
3,90 + 2,full-time: barcelona 0-3 bayern munich,NaN,NaN,NaN,timer,barcelona,BAR,bayern munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...",barcelona (3-1-4-2): marc-andre ter stegen; ro...,"bayern subs: omar ricards, serge gnabry, kings...",bayern (4-2-3-1): manuel neuer; benjamin pavar...
4,90,there will be two minutes of added time.,NaN,NaN,NaN,timer,barcelona,BAR,bayern munchen,FCB,0 - 3,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...",barcelona (3-1-4-2): marc-andre ter stegen; ro...,"bayern subs: omar ricards, serge gnabry, kings...",bayern (4-2-3-1): manuel neuer; benjamin pavar...


In [300]:
df_new['away_team_formation'] = df_new['away_team_formation_info'].apply(lambda x: re.findall(r'\(.*?\)', x)[0])
df_new['home_team_formation'] = df_new['home_team_formation_info'].apply(lambda x: re.findall(r'\(.*?\)', x)[0])

In [301]:
df_new['home_team_formation_info'] = df_new['home_team_formation_info'].apply(lambda x: x.replace(";", ","))
df_new['away_team_formation_info'] = df_new['away_team_formation_info'].apply(lambda x: x.replace(";", ","))
df_new['home_team_subs_info'] = df_new['home_team_subs_info'].apply(lambda x: x.replace(";", ","))
df_new['away_team_subs_info'] = df_new['away_team_subs_info'].apply(lambda x: x.replace(";", ","))

In [302]:
df_new

,time,comment,event,event_player,event_team,comment_desc,home_team,home_team_abbr,away_team,away_team_abbr,...,match,date,link,match_id,home_team_subs_info,home_team_formation_info,away_team_subs_info,away_team_formation_info,away_team_formation,home_team_formation
0,NaN,thanks for joining our commentary this evening...,NaN,NaN,NaN,full time summary,barcelona,BAR,bayern munchen,FCB,...,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...","barcelona (3-1-4-2): marc-andre ter stegen, ro...","bayern subs: omar ricards, serge gnabry, kings...","bayern (4-2-3-1): manuel neuer, benjamin pavar...",(4-2-3-1),(3-1-4-2)
1,NaN,barcelona are next in action at home in laliga...,NaN,NaN,NaN,full time summary,barcelona,BAR,bayern munchen,FCB,...,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...","barcelona (3-1-4-2): marc-andre ter stegen, ro...","bayern subs: omar ricards, serge gnabry, kings...","bayern (4-2-3-1): manuel neuer, benjamin pavar...",(4-2-3-1),(3-1-4-2)
2,NaN,bayern munich have eased past barcelona in the...,NaN,NaN,NaN,full time summary,barcelona,BAR,bayern munchen,FCB,...,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...","barcelona (3-1-4-2): marc-andre ter stegen, ro...","bayern subs: omar ricards, serge gnabry, kings...","bayern (4-2-3-1): manuel neuer, benjamin pavar...",(4-2-3-1),(3-1-4-2)
3,90 + 2,full-time: barcelona 0-3 bayern munich,NaN,NaN,NaN,timer,barcelona,BAR,bayern munchen,FCB,...,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...","barcelona (3-1-4-2): marc-andre ter stegen, ro...","bayern subs: omar ricards, serge gnabry, kings...","bayern (4-2-3-1): manuel neuer, benjamin pavar...",(4-2-3-1),(3-1-4-2)
4,90,there will be two minutes of added time.,NaN,NaN,NaN,timer,barcelona,BAR,bayern munchen,FCB,...,Barcelona vs Bayern Muenchen,9/14/21,https://www.goal.com/en/match/barcelona-vs-bay...,0,"barcelona subs: sergino dest, riqui puig, yusu...","barcelona (3-1-4-2): marc-andre ter stegen, ro...","bayern subs: omar ricards, serge gnabry, kings...","bayern (4-2-3-1): manuel neuer, benjamin pavar...",(4-2-3-1),(3-1-4-2)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9490,NaN,"real madrid subs: david alaba, jesus vallejo, ...",NaN,NaN,NaN,half time summary,real madrid,RMA,manchester city,MCI,...,Real madrid Vs Manchester City,5/4/22,https://www.goal.com/en/match/real-madrid-vs-m...,123,"real madrid subs: david alaba, jesus vallejo, ...","real madrid (4-3-3): thibaut courtois, dani ca...","manchester city subs: nathan ake, raheem sterl...","manchester city (4-3-3): ederson, kyle walker,...",(4-3-3),(4-3-3)
9491,NaN,real madrid (4-3-3): thibaut courtois; dani ca...,NaN,NaN,NaN,half time summary,real madrid,RMA,manchester city,MCI,...,Real madrid Vs Manchester City,5/4/22,https://www.goal.com/en/match/real-madrid-vs-m...,123,"real madrid subs: david alaba, jesus vallejo, ...","real madrid (4-3-3): thibaut courtois, dani ca...","manchester city subs: nathan ake, raheem sterl...","manchester city (4-3-3): ederson, kyle walker,...",(4-3-3),(4-3-3)
9492,NaN,manchester city will be looking for another sh...,NaN,NaN,NaN,half time summary,real madrid,RMA,manchester city,MCI,...,Real madrid Vs Manchester City,5/4/22,https://www.goal.com/en/match/real-madrid-vs-m...,123,"real madrid subs: david alaba, jesus vallejo, ...","real madrid (4-3-3): thibaut courtois, dani ca...","manchester city subs: nathan ake, raheem sterl...","manchester city (4-3-3): ederson, kyle walker,...",(4-3-3),(4-3-3)
9493,NaN,real madrid enter the second leg trailing city...,NaN,NaN,NaN,half time summary,real madrid,RMA,manchester city,MCI,...,Real madrid Vs Manches

In [ ]:
for i in range(len(df_new)):
    
    df_new['home_team_formation_info']